In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import warnings
import sys

import data_preprocess as dp # data_preprocess.py

if not sys.warnoptions:
    warnings.simplefilter("ignore")

# Solar Energy Production Forecasting

We are creating a model that can predict solar energy production for every hour of the next day

TODO: 
- [ ] Explore the dataset
- [ ] More than one prediction model
- [ ] Feature engineering 
- [ ] Model interpretation
- [ ] Create short notebook


# Data exploration

- Search domain knowledge 
- Check if the data is intuitive
- Explore individual features
- Explore pairs and groups of features
- Clean up features

## Datapreprocessing

The `data_preprocess()` function performs several operations to prepare datasets for machine learning analysis. First, it verifies if all the required data files exist on disk and then reads these parquet files into Pandas DataFrames. Afterward, it concatenates and merges multiple subsets of data based on location and time, performing some feature engineering like calculating time differences and labeling data as either "observed" or "estimated". Finally, it encodes the 'location' column of the DataFrames into numerical values using LabelEncoder from scikit-learn, and returns two DataFrames: one with features (`X_train`) and another with both features and targets (`X_train_with_targets`).

This is what we have considered in this section:
- Handle Missing Values ( `data_preprocess()`)
- Feature Engineering ( `data_preprocess()`)
- Normalization 

In [ ]:
X_train, X_train_with_targets = dp.data_preprocess()
X_train.head()

## Feature selection

Here we are going to look further into how to select the best features for our model. Initially we will begin with looking at correleations between the features and the target variable. This with the help of scatterplots, heatmaps and correlation matrices.

### Correlations

We decieded to look closer into the features and their correleations with the target variable `pv_measurement`. 

In [ ]:
corr = X_train_with_targets.drop(columns=["time", "date_calc"], inplace=False).corr()
corr["pv_measurement"].sort_values(ascending=False)

The features where the Corr > |0.1| are the ones we find most interesting. 

#### Positive correlations

Here we can also see that `direct_rad:W`, `direct_rad_1h:J`, `clear_sky_rad:W`,`clear_sky_energy_1h:J`, `diffuse_rad:W`, `diffuse_rad_1h:J` and `sun_elevation:d` are highly correlated with each other. However given the feature name there is not that much information about the feature itself. Therefore we have decieded to look closer into the features and how they effect the power production of the solar panel.

1. **Direct radiation (W)**: This parameter measures the amount of direct sunlight that reaches the ground. Direct radiation is the most effective type of sunlight for solar energy production, so a higher value would generally result in more energy being produced by photovoltaic cells.

2. **Direct radiation 1h (J)**: Similar to direct_rad, this parameter measures the total amount of direct radiation energy within a one-hour period. This could be useful for assessing the potential for solar energy production over short periods of time, especially when direct sunlight is available.

3. **Clear sky radiation (W)**: This measures the solar radiation that would reach the ground under clear sky conditions. This is a theoretical maximum amount of solar radiation, assuming no clouds or other atmospheric interference. It serves as a benchmark to gauge how much energy could be produced under optimal conditions.

4. **Clear sky energy 1h (J)**: This parameter measures the potential energy production under clear sky conditions within a one-hour timeframe. A higher value would indicate more solar energy is available for capture. This would be a direct influence on the solar panel's efficiency, assuming no other environmental issues like dirt or dust on the panels.

5. **Diffuse radiation (W)**: This measures the amount of diffuse radiation, which is sunlight scattered by particles in the atmosphere. Unlike direct sunlight, diffuse radiation comes from all directions. Solar panels can still capture this type of radiation, although typically at a lower efficiency compared to direct sunlight.

6. **Diffuse radiation 1h (W)**: Similar to diffuse_rad, but it measures the total diffuse radiation energy within a one-hour period. This could be useful for understanding the variability in power generation over short periods of time, as diffuse radiation can be influenced by factors like cloud cover and atmospheric conditions.

7. **Sun elevation (d)**: This is the angle of the sun in relation to an observer's horizon. A higher angle (closer to 90 degrees) means the sun is directly overhead, which is generally more favorable for solar energy production. A lower angle means less direct sunlight and thus lower potential for energy generation. This parameter would be particularly important in regions with large changes in sun elevation throughout the year, such as high-latitude areas.

Using these features in a predictive model can help the model to better understand and forecast solar energy production. Each of these four features captures a different aspect of the environmental conditions that influence solar energy capture, and collectively they can provide a comprehensive view of solar energy potential. Understanding how these variables interact and influence solar energy output can be crucial for optimizing the efficiency of solar energy systems.

Therefore we have decieded to create different scatterplots given the features with the highest correlation and the target variable, in order to hopefully see a pattern.

In [ ]:
features = ["direct_rad:W", "clear_sky_rad:W", "sun_elevation:d", "is_in_shadow:idx", "diffuse_rad_1h:J", "t_1000hPa:K"] #Note pv_measurement not a feature
fig, axes = plt.subplots(3, 2, figsize=(12, 12))
flat_axes = axes.flatten()
for i, feature in enumerate(features):
    ax = flat_axes[i]
    sns.scatterplot(data=X_train_with_targets, x=feature, y="pv_measurement", hue="location", ax=ax)
plt.tight_layout()
plt.show()

It's difficult to get a complete overview of the data, but we can see that the power production is highly correlated with the direct radiation and the clear sky radiation AND that the location matters. This makes sense, as the power production is dependent on the amount of radiation hitting the solar panel. We can see that location 0 or A is the one with the highest power production in every plot.

#### Negative correlations

Now we want to look closer into the features which are negatively correlated with the target variable:  `relative_humidity_1000hPa:p`, `air_density_2m:kgm3`, `location` and `is_in_shadow:idx`. 

1. **Relative Humidity 1000hPa (p)**: This parameter measures the moisture content in the air at a specific pressure level (1000 hPa). While humidity itself doesn't directly affect the efficiency of solar panels, it can influence cloud formation and atmospheric conditions. High humidity could potentially lead to cloud cover, which would reduce the amount of solar radiation reaching the ground.

2. **Air Density 2m (kg/m³)**: This measures the density of the air 2 meters above the ground. Air density can affect the transmission of sunlight, albeit minimally. Lower air density could mean less scattering and absorption of sunlight, potentially leading to slightly increased solar energy production.

3. **Location**: The geographical location is crucial for solar energy production. It dictates many other factors such as the angle of sunlight, average weather conditions, and duration of daylight. For example, locations closer to the equator generally receive more direct sunlight, increasing the potential for solar energy production.

4. **Is in Shadow (idx)**: This index  indicates whether a location is currently in shadow, perhaps due to buildings, trees, or other obstructions. Being in the shadow would significantly reduce solar radiation reaching the solar panels, thus reducing energy production. This would be an immediate and significant factor affecting solar energy output.

In [ ]:
features = ["relative_humidity_1000hPa:p", "air_density_2m:kgm3", "location", "is_in_shadow:idx"]

fig, axes = plt.subplots(2, 2, figsize=(12, 12))
flat_axes = axes.flatten()
for i, feature in enumerate(features):
    ax = flat_axes[i]
    sns.scatterplot(data=X_train_with_targets, x=feature, y="pv_measurement", hue="location", ax=ax)
plt.tight_layout()
plt.show()

None of the negative correlation plots gave any useful information, other than what we saw in the positive correlation plots.

#### Heatmaps

Can be great for showing correlation between multiple variables at once, which means we might see some correlations between other features that we didn't see in the scatterplots.

In [ ]:
# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

# Set up the matplotlib figure
fig, ax = plt.subplots(figsize=(15, 12))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

plt.title('Feature Correlation Heatmap')
plt.show()

In the end we have decieded to move forward with the features which has a correleation of higher than |0.138989|

In [ ]:
# Calculate absolute correlations
abs_corr = corr['pv_measurement'].abs()

# Filter by absolute value greater than or equal to 0.138989
features = abs_corr[abs_corr >= 0.138989].index.tolist()

# Remove 'pv_measurement' if it's in the list as it's the target
if 'pv_measurement' in features:
    features.remove('pv_measurement')

print("Filtered features with absolute correlation greater than or equal to 0.138989:", features)
X_train = X_train.loc[:, features]

X_train.info()

## Prediction model 1 

First attempt: a simple decision tree model using sklearn.

In [ ]:
from data_preprocess import data_preprocess, get_training_data, get_input_data, prepare_submission
import pandas as pd
import numpy as np
import warnings
import sys

if not sys.warnoptions:
    warnings.simplefilter("ignore")

In [ ]:
features = ["direct_rad:W", "direct_rad_1h:J", "clear_sky_rad:W", "clear_sky_energy_1h:J", "diffuse_rad:W", "sun_elevation:d","is_day:idx", 
            "is_in_shadow:idx", "diffuse_rad_1h:J", "t_1000hPa:K","relative_humidity_1000hPa:p", "air_density_2m:kgm3", "A", "B", "C"]

all_x_train = data_preprocess(one_hot_location=True)
X_train, targets = get_training_data(all_x_train, features)

In [ ]:
# train the decision tree model

from sklearn import tree

X = X_train.values
y = targets

clf = tree.DecisionTreeRegressor()
clf = clf.fit(X, y)

In [ ]:
X_test = get_input_data()
predictions = clf.predict(X_test[features].values)

In [ ]:
submission = prepare_submission(X_test, predictions)
submission.to_csv('submissions/simple_tree_submission.csv', index=False)

Trying a gradient booster reressor just for laughs.

In [ ]:
features = ["direct_rad:W", "direct_rad_1h:J", "clear_sky_rad:W", "clear_sky_energy_1h:J", "diffuse_rad:W", "sun_elevation:d","is_day:idx", 
            "is_in_shadow:idx", "diffuse_rad_1h:J", "t_1000hPa:K","relative_humidity_1000hPa:p", "air_density_2m:kgm3", "A", "B", "C"]

all_x_train = data_preprocess(one_hot_location=True)
X_train, targets = get_training_data(all_x_train, features)

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split

X = X_train.values
y = targets

reg = GradientBoostingRegressor(random_state=0)
reg.fit(X, y)

In [ ]:
X_test = get_input_data()
predictions = reg.predict(X_test[features].values)

In [ ]:
submission = prepare_submission(X_test, predictions)
submission.to_csv('submissions/grad_boost_regressor.csv', index=False)

and using a NN

In [ ]:
features = ["direct_rad:W", "direct_rad_1h:J", "clear_sky_rad:W", "clear_sky_energy_1h:J", "diffuse_rad:W", "sun_elevation:d","is_day:idx", 
            "is_in_shadow:idx", "diffuse_rad_1h:J", "t_1000hPa:K","relative_humidity_1000hPa:p", "air_density_2m:kgm3", "A", "B", "C"]

all_x_train = data_preprocess(one_hot_location=True)
X_train, targets = get_training_data(all_x_train, features)

In [ ]:
from sklearn.neural_network import MLPRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split

X = X_train.values
y = targets

regr = MLPRegressor(random_state=1, max_iter=500).fit(X, y)

X_test = get_input_data()
predictions = regr.predict(X_test[features].values)

submission = prepare_submission(X_test, predictions)
submission.to_csv('submissions/mlpgr_regressor.csv', index=False)

## Prediction model 2

In [ ]:
## TODO CODE HERE

## Model interpretation

In [ ]:
## TODO CODE HERE